In [1]:
import typing
import datetime
import requests

import dotenv
import pydantic
import rich

In [2]:
import cltrier_lib as lib

In [3]:
ENV = dotenv.dotenv_values()

In [4]:
REQUEST_ENDPOINT: str = ENV["OLLAMA_ENDPOINT"]
REQUEST_META: typing.Dict = {
    "headers": {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {ENV["OLLAMA_BEARER"]}"
    }
}

In [5]:
class BaseResponse(pydantic.BaseModel):
    model: str

    created_at: datetime.datetime
    total_duration: datetime.timedelta

    prompt_eval_count: int
    eval_count: int

    @pydantic.field_validator("total_duration", mode="before")
    def convert_to_milliseconds(cls, val: int):
        return val * 1e-9

In [6]:
class GenerateResponse(BaseResponse):
    response: str

In [7]:
class ChatResponse(BaseResponse):
    message: lib.inference.schemas.Message

In [8]:
rich.print(GenerateResponse(**requests.post(
    f"{REQUEST_ENDPOINT}/generate/",
    json={
        "model": "llama3.1:8b-instruct-q6_K",
        "prompt": "Why is the sky blue?",
        "stream": False
    },
    **REQUEST_META
).json()))

GenerateResponse(
    model='llama3.1:8b-instruct-q6_K',
    created_at=datetime.datetime(2024, 9, 10, 8, 14, 55, 938160, tzinfo=TzInfo(UTC)),
    total_duration=datetime.timedelta(seconds=5, microseconds=60597),
    prompt_eval_count=17,
    eval_count=395,
    response="The sky appears blue to us because of a phenomenon called Rayleigh scattering, named after the 
British physicist Lord Rayleigh. Here's what happens:\n\n**Short answer:** The shorter (blue) wavelengths are 
scattered more than longer (red) wavelengths by the tiny molecules of gases in the Earth's 
atmosphere.\n\n**Detailed explanation:**\n\nWhen sunlight enters our atmosphere, it encounters tiny molecules of 
gases such as nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light. When 
light interacts with these molecules, they scatter the light in all directions.\n\nNow, here's the key part: 
shorter wavelengths (like blue and violet) are scattered more than longer wavelengths (like red and orange). This 
is because the smaller molecules are more effective at scattering the shorter wavelengths. As a result, the blue 
light is distributed throughout the atmosphere, giving it a uniform color.\n\n**But why not all colors?**\n\nThere 
are a few reasons why we don't see other colors scattered equally:\n\n1. **Wavelength:** The shorter wavelengths 
(blue and violet) are more easily scattered than longer wavelengths.\n2. **Atmospheric gases:** Other molecules in 
the atmosphere, like ozone (O3), absorb certain wavelengths of light, making them less visible to us.\n3. 
**Sunlight composition:** Sunlight is composed mostly of blue and yellow light, which are relatively easy to 
scatter.\n\n**What about at sunrise and sunset?**\n\nDuring these times, the sun's rays travel through a thicker 
layer of atmosphere before reaching our eyes. This scatters more of the shorter wavelengths (blue and violet), 
leaving mainly longer wavelengths (red and orange) to reach us. That's why we often see those beautiful hues during
dawn and dusk!\n\nIn summary: the sky appears blue because of Rayleigh scattering, where shorter wavelengths (like 
blue) are scattered more than longer wavelengths by tiny atmospheric molecules.\n\nHow was that? Did I help you 
understand this phenomenon better?"
)

In [9]:
rich.print(ChatResponse(**requests.post(
    f"{REQUEST_ENDPOINT}/chat/",
    json={
        "model": "llama3.1:8b-instruct-q6_K",
        **lib.inference.schemas.Chat(messages=[
            lib.inference.schemas.Message(
                role="user",
                content="why is the sky blue?"
            )
        ]).model_dump(),
        "stream": False
    },
    **REQUEST_META
).json()))

ChatResponse(
    model='llama3.1:8b-instruct-q6_K',
    created_at=datetime.datetime(2024, 9, 10, 8, 15, 1, 556225, tzinfo=TzInfo(UTC)),
    total_duration=datetime.timedelta(seconds=5, microseconds=485905),
    prompt_eval_count=17,
    eval_count=434,
    message=Message(
        role='assistant',
        content="The sky appears blue to us during the day due to a phenomenon called Rayleigh scattering, named 
after the British physicist Lord Rayleigh who first described it in the late 19th century. Here's why:\n\n1. 
**Sunlight and Its Spectrum**: Sunlight is made up of various wavelengths (colors) of light, which are all mixed 
together. These include visible colors such as red, orange, yellow, green, blue, indigo, and violet, plus 
ultraviolet (UV) and infrared (IR) rays that we cannot see.\n\n2. **Scattering**: When sunlight enters Earth's 
atmosphere, it encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2). These molecules scatter the 
light in all directions but are much more effective at scattering shorter wavelengths than longer ones.\n\n3. 
**Rayleigh Scattering Effect**: The key here is that Rayleigh scattering favors shorter wavelengths over longer 
wavelengths. Blue light, being on the shorter end of the visible spectrum, is scattered more than red light. This 
means that when we look up at the sky, we see primarily blue light, which has been scattered in all directions by 
the atmosphere.\n\n4. **Why Not All Parts of the Sky Appear Blue**: The color of the sky can vary due to several 
factors including:\n   - **Atmospheric Conditions**: During sunrise and sunset, the shorter wavelengths (like blue)
are scattered away more efficiently by atmospheric particles such as dust, pollen, and water vapor, making the sun 
appear red.\n   - **Clouds and Pollution**: Clouds and air pollution can scatter light in a way that makes the sky 
look gray or white. Thick clouds can block sunlight and prevent any color from appearing, leading to a dark blue or
black appearance during nighttime.\n   - **Atmospheric Attenuation**: The Earth's atmosphere attenuates (weakens) 
some wavelengths more than others as they pass through it. This is why distant objects often appear red.\n\nIn 
summary, the sky appears blue because of the scattering effect caused by shorter wavelengths in sunlight 
interacting with the tiny molecules of gases in the atmosphere, a phenomenon known as Rayleigh scattering."
    )
)